In [92]:
!pip install pymysql
!sudo apt-get install -y fonts-nanum # 나눔 폰트 설치 (설치 후 런타임 재시작하고 코드 실행)
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory

In [93]:
import itertools
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib import rc
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import time
import statsmodels.api as sm
from glob import glob

warnings.filterwarnings(action='ignore')
mpl.rcParams['axes.unicode_minus'] = False
rc('font', family='NanumBarunGothic')

endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema)
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

def load_data_from_rds(tabel_name):
  sql = "SELECT * FROM {}".format(tabel_name)
  df = pd.read_sql(sql, db_connection)
  return df

In [94]:
def preprocess_df(df, onehot_col=None, scaling_col=None, drop_col = None, labeling_col = None, scaling_func=MinMaxScaler) :
    """
    :param df: 사용할 데이터프레임
    :param onehot_col: default = None, 원핫인코딩할 컬럼 리스트
    :param scaling_col: default = None, 스케일링할 컬럼 리스트
    :param drop_col: default = None, 사용하지 않을 컬럼 리스트
    :param labeling_col: default = None, 라벨링이 필요한 컬럼 리스트
    :param scaling_func: 스케일링시 사용할 컬럼, MinMaxScaler or StandardScaler
    :return: 전처리 된 데이터 프레임
    """
    if drop_col is not None :
        df = df.drop(drop_col, axis=1)
    if onehot_col is not None:
        df = pd.get_dummies(df, colums=onehot_col)
    if scaling_col is not None :
        for c in scaling_col :
            scaler = scaling_func()
            scaler.fit(df[[c]])
            df[c] = scaler.transform(df[[c]])
    if labeling_col is not None :
        for c in labeling_col :
            encoder = LabelEncoder()
            encoder.fit(df[c])
            df[c] = encoder.transform(df[c])
    return df

In [95]:
# GridSearchCV의 최적 하이퍼파라미터로 학습된 Estimator 반환하는 함수
def optimizing_lr(clf, train_x, train_y):
  parameters = {'penalty': ['l1','l2','elasticnet','none'],
                'C': np.arange(0,1,0.1),
                'random_state': range(3,7,2)}
  start = time.time()
  grid_cv = GridSearchCV(clf, param_grid=parameters, n_jobs=-1, scoring='accuracy', cv=5)
  grid_cv.fit(train_x, train_y)

  #최적의 하이퍼 파라미터 출력
  print('GridSearch 수행시간 : ', time.time()-start)
  print('최적 하이퍼 파라미터 :',grid_cv.best_params_)

  return grid_cv.best_estimator_

In [96]:
def logistic_regression(train_x, train_y, feature_importance = False, optimizing = False):

  # logistic regression
  lr_clf = LogisticRegression()

  # optimizing parameters
  if optimizing :
    lr_clf = optimizing_lr(lr_clf, train_x, train_y)
  else:
    lr_clf.fit(train_x, train_y)

  return lr_clf

In [97]:
# RFECV로 선택된 변수를 반환하는 함수
def rfecv_lr(clf, train_x, train_y):
  min_features_to_select=1
  rfe_cv = RFECV(clf, step=1, cv=5, min_features_to_select=min_features_to_select)
  rfe_cv.fit(train_x, train_y)

  # 최적의 변수 개수 출력
  print("Optimal number of features : %d" % rfe_cv.n_features_)

  # plt.figure()
  # plt.xlabel("Number of features selected")
  # plt.ylabel("Cross validation score (accuracy)")
  # plt.plot(
  #   range(min_features_to_select, len(rfe_cv.grid_scores_) + min_features_to_select),
  #   rfe_cv.grid_scores_,)
  # plt.show()

  return train_x.columns[rfe_cv.support_]

In [98]:
def evalutation_lf(test_y, pred_y):
  # 평가
  acc = accuracy_score(test_y, pred_y)
  f1 = f1_score(test_y, pred_y, average='macro')

  result = pd.DataFrame([['Logistic Regression RFECV', acc, f1]],
                         columns = ['Model', 'Accuracy', 'F1Score'])
  display(result)

In [99]:
def summary_lr(x, y):
  con_x = sm.add_constant(x)
  model = sm.OLS(y, con_x)
  result = model.fit()
  print(result.summary())

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
overpass_list = glob('/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/*overpass*.csv')
overpass_list = [p.replace('\\', '/') for p in overpass_list]
print(overpass_list)
tunnel_list = glob('/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/*tunnel*.csv')
tunnel_list = [p.replace('\\', '/') for p in tunnel_list]
print(tunnel_list)

['/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn2_median.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn5_drop.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn3_median.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn5_median.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_mean_drop.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_mean_median.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn2_drop.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn1_drop.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn3_drop.csv', '/content/drive/MyDrive/Colab Notebooks/데청캠_2팀/MainData/prepro/overpass_knn4_median.csv', '/content/drive/MyDrive/Colab Notebooks

In [102]:
onehot_col = ['지형기호(2.3km)', '최우점식물군락']
scaling_col = ['폭(m)', '연장(m)', '주변 로드킬 빈도', '산책로까지의 최단 거리(km)', '농가까지의 거리(km)', '도로 최고제한속도(km/h)',
               '하천거리(km)', '주변 동물종 개수', '등산로까지 최단거리(km)', '유도울타리_연장_m', '유도울타리_높이_m', '교통량', '건물까지거리(km)']
drop_col = ['번호', '위도', '경도', '생태통로_유형', '이용확인종_수', '유도울타리_위도_2.3km', '유도울타리_경도_2.3km', '식물군락명']
scaling_col_tunnel = scaling_col + ['높이(m)', '개방도']

# 육교형

In [103]:
for case in overpass_list :
  df = pd.read_csv(case, encoding='cp949')
  name = case.split('/')[-1].split('.')[0]
  df = preprocess_df(df, drop_col=drop_col, labeling_col=onehot_col, scaling_col=scaling_col, scaling_func=StandardScaler)
 
  # x, y
  x=df.drop('생태통로_효율성', axis=1)
  y=df['생태통로_효율성']

  # train & test set
  train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=df['생태통로_효율성'])

  # logistic regression
  lr_clf =logistic_regression(train_x, train_y, optimizing=True)
  pred_y = lr_clf.predict(test_x)

  # evaluation
  evalutation_lf(test_y, pred_y)

  # summary
  summary_lr(train_x, train_y)

  # rfecv
  selected_x = rfecv_lr(lr_clf, train_x, train_y)

  # RFECV로 선택된 변수로 logistic regression
  lr_clf = logistic_regression(train_x[selected_x], train_y, optimizing=True)
  pred_y = lr_clf.predict(test_x[selected_x])

  # evaluation
  evalutation_lf(test_y, pred_y)

  # summary
  summary_lr(train_x[selected_x], train_y)


GridSearch 수행시간 :  13.121912240982056
최적 하이퍼 파라미터 : {'C': 0.5, 'penalty': 'l2', 'random_state': 3}


,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.533333,0.414656


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     2.119
Date:                Fri, 19 Aug 2022   Prob (F-statistic):            0.00457
Time:                        09:04:54   Log-Likelihood:                -220.32
No. Observations:                 240   AIC:                             482.6
Df Residuals:                     219   BIC:                             555.7
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.0622      0.252  

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.666667,0.487356


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.154
Date:                Fri, 19 Aug 2022   Prob (F-statistic):              0.284
Time:                        09:05:04   Log-Likelihood:                -240.97
No. Observations:                 240   AIC:                             485.9
Df Residuals:                     238   BIC:                             492.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9423      0.043     22.010      0.0

...

                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                 0.0003641
Date:                Fri, 19 Aug 2022   Prob (F-statistic):              0.985
Time:                        09:07:14   Log-Likelihood:                -182.86
No. Observations:                 146   AIC:                             369.7
Df Residuals:                     144   BIC:                             375.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9071      0.171      5.300      0.0

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.65,0.579249


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     2.133
Date:                Fri, 19 Aug 2022   Prob (F-statistic):            0.00426
Time:                        09:07:21   Log-Likelihood:                -220.19
No. Observations:                 240   AIC:                             482.4
Df Residuals:                     219   BIC:                             555.5
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.2849      0.241  

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.6,0.548084


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     6.565
Date:                Fri, 19 Aug 2022   Prob (F-statistic):           0.000279
Time:                        09:07:28   Log-Likelihood:                -231.93
No. Observations:                 240   AIC:                             471.9
Df Residuals:                     236   BIC:                             485.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9503      0.041     22.917   

# 터널형

In [104]:
for case in tunnel_list :
  df = pd.read_csv(case, encoding='cp949')
  name = case.split('/')[-1].split('.')[0]
  df = preprocess_df(df, drop_col=drop_col, labeling_col=onehot_col, scaling_col=scaling_col, scaling_func=StandardScaler)
 
  # x, y
  x=df.drop('생태통로_효율성', axis=1)
  y=df['생태통로_효율성']

  # train & test set
  train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=df['생태통로_효율성'])

  # logistic regression
  lr_clf =logistic_regression(train_x, train_y, optimizing=True)
  pred_y = lr_clf.predict(test_x)

  # evaluation
  evalutation_lf(test_y, pred_y)

  # summary
  summary_lr(train_x, train_y)

  # rfecv
  selected_x = rfecv_lr(lr_clf, train_x, train_y)

  # RFECV로 선택된 변수로 logistic regression
  lr_clf = logistic_regression(train_x[selected_x], train_y, optimizing=True)
  pred_y = lr_clf.predict(test_x[selected_x])

  # evaluation
  evalutation_lf(test_y, pred_y)

  # summary
  summary_lr(train_x[selected_x], train_y)

GridSearch 수행시간 :  6.430907249450684
최적 하이퍼 파라미터 : {'C': 0.7000000000000001, 'penalty': 'l2', 'random_state': 3}


,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.52,0.398551


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.398
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     2.356
Date:                Fri, 19 Aug 2022   Prob (F-statistic):            0.00363
Time:                        09:07:34   Log-Likelihood:                -102.59
No. Observations:                  97   AIC:                             249.2
Df Residuals:                      75   BIC:                             305.8
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.6758      1.489  

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.56,0.412698


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     14.30
Date:                Fri, 19 Aug 2022   Prob (F-statistic):           0.000273
Time:                        09:07:39   Log-Likelihood:                -120.37
No. Observations:                  97   AIC:                             244.7
Df Residuals:                      95   BIC:                             249.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1837      0.113     10.496      0.0

...

                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     14.63
Date:                Fri, 19 Aug 2022   Prob (F-statistic):           0.000234
Time:                        09:09:59   Log-Likelihood:                -120.22
No. Observations:                  97   AIC:                             244.4
Df Residuals:                      95   BIC:                             249.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1975      0.115     10.459      0.0

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.64,0.476383


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     2.262
Date:                Fri, 19 Aug 2022   Prob (F-statistic):            0.00534
Time:                        09:10:06   Log-Likelihood:                -103.37
No. Observations:                  97   AIC:                             250.7
Df Residuals:                      75   BIC:                             307.4
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.2676      1.496  

,Model,Accuracy,F1Score
0,Logistic Regression RFECV,0.6,0.44958


                            OLS Regression Results                            
Dep. Variable:               생태통로_효율성   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     3.455
Date:                Fri, 19 Aug 2022   Prob (F-statistic):           0.000374
Time:                        09:10:15   Log-Likelihood:                -107.71
No. Observations:                  97   AIC:                             241.4
Df Residuals:                      84   BIC:                             274.9
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.3599      0.973  